In [35]:
import os
import os.path as path
import gc
import re
import math
import json

In [36]:
import numpy as np
import pandas as pd

In [37]:
import matplotlib.pyplot as plt
import seaborn as sns

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [39]:
pd.set_option('display.max_row', 50)
pd.set_option('display.max_columns', 100)

In [40]:
# 디렉토리 기본 경로 설정
DIR_PATH = path.join('..', 'data', 'sql_dummy')
DIR_SAVE_PATH = path.join('..', 'data', 'output')

print(DIR_PATH)
print(DIR_SAVE_PATH)

..\data\sql_dummy
..\data\output


In [41]:
# sql_bean.csv
bean_read = pd.read_csv(path.join(DIR_PATH, 'sql_bean.csv'), low_memory=False, encoding='cp949')

print(bean_read.shape)
bean_read.head()

(47, 8)


,idx,created_date,updated_date,name_ko,name_en,summary,thumbnail,user_grade
0,1,NaN,NaN,에티오피아 예가체프 G2 워시드 (중배전),에티오피아 예가체프 G2 워시드 (중배전),아이템 요약 영역 입니다!!!,default_bean.png,0
1,2,NaN,NaN,과테말라 안티구아 SHB 워시드 (강배전),과테말라 안티구아 SHB 워시드 (강배전),아이템 요약 영역 입니다!!!,default_bean.png,0
2,3,NaN,NaN,에티오피아 코케 G1 펄프드내추럴 (중배전),에티오피아 코케 G1 펄프드내추럴 (중배전),아이템 요약 영역 입니다!!!,default_bean.png,0
3,4,NaN,NaN,케냐 키암부 AA 워시드 (중배전),케냐 키암부 AA 워시드 (중배전),아이템 요약 영역 입니다!!!,default_bean.png,0
4,5,NaN,NaN,콜롬비아 콜롬비아 SUPREMO 워시드 (중배전),콜롬비아 콜롬비아 SUPREMO 워시드 (중배전),아이템 요약 영역 입니다!!!,default_bean.png,0


In [42]:
# sql_bean_score.csv
bean_score_read = pd.read_csv(path.join(DIR_PATH, 'sql_bean_score.csv'), low_memory=False, encoding='cp949')

print(bean_score_read.shape)
bean_score_read.head()

(47, 12)


,idx,created_date,updated_date,flavor,acidity,sweetness,bitterness,body,balance,roasting_point,coffeeing_note,bean_idx
0,1,NaN,NaN,7,7,6,4,4,5,중배전,"꽃, 과일, 부드러운, 플로럴, 레몬, 허니",1
1,2,NaN,NaN,6,3,7,7,7,5,강배전,"스모크, 우아한, 중후한, 블랙커런트, 갈색설탕, 다크초코",2
2,3,NaN,NaN,8,8,7,4,5,5,중배전,"베리, 체리, 허니",3
3,4,NaN,NaN,6,6,7,6,7,5,중배전,"자몽, 당밀, 카라멜",4
4,5,NaN,NaN,7,5,6,6,6,5,중배전,"마일드, 적포도, 메이플시럽, 다크초코",5


In [44]:
# sql_bean_detail.csv
bean_detail_read = pd.read_csv(path.join(DIR_PATH, 'sql_bean_detail.csv'), low_memory=False, encoding='cp949')

print(bean_detail_read.shape)
bean_detail_read.head()

(47, 10)


,idx,created_date,updated_date,decaffeination,description,origin,region,rank,processing,bean_idx
0,1,NaN,NaN,F,상세 내용 입니다!!!,에티오피아,예가체프,G2,워시드,1
1,2,NaN,NaN,F,상세 내용 입니다!!!,과테말라,안티구아,SHB,워시드,2
2,3,NaN,NaN,F,상세 내용 입니다!!!,에티오피아,코케,G1,펄프드내추럴,3
3,4,NaN,NaN,F,상세 내용 입니다!!!,케냐,키암부,AA,워시드,4
4,5,NaN,NaN,F,상세 내용 입니다!!!,콜롬비아,콜롬비아,SUPREMO,워시드,5


In [47]:
bean_data = bean_read.copy()
bean_data = pd.merge(bean_data, bean_detail_read.drop(['idx', 'created_date', 'updated_date'], axis=1), how='left', left_on='idx', right_on='bean_idx')
bean_data.drop('bean_idx', axis=1, inplace=True)
bean_data = pd.merge(bean_data, bean_score_read.drop(['idx', 'created_date', 'updated_date'], axis=1), how='left', left_on='idx', right_on='bean_idx')
bean_data.drop('bean_idx', axis=1, inplace=True)
print(bean_data.shape)
bean_data.head()

(47, 22)


,idx,created_date,updated_date,name_ko,name_en,summary,thumbnail,user_grade,decaffeination,description,origin,region,rank,processing,flavor,acidity,sweetness,bitterness,body,balance,roasting_point,coffeeing_note
0,1,NaN,NaN,에티오피아 예가체프 G2 워시드 (중배전),에티오피아 예가체프 G2 워시드 (중배전),아이템 요약 영역 입니다!!!,default_bean.png,0,F,상세 내용 입니다!!!,에티오피아,예가체프,G2,워시드,7,7,6,4,4,5,중배전,"꽃, 과일, 부드러운, 플로럴, 레몬, 허니"
1,2,NaN,NaN,과테말라 안티구아 SHB 워시드 (강배전),과테말라 안티구아 SHB 워시드 (강배전),아이템 요약 영역 입니다!!!,default_bean.png,0,F,상세 내용 입니다!!!,과테말라,안티구아,SHB,워시드,6,3,7,7,7,5,강배전,"스모크, 우아한, 중후한, 블랙커런트, 갈색설탕, 다크초코"
2,3,NaN,NaN,에티오피아 코케 G1 펄프드내추럴 (중배전),에티오피아 코케 G1 펄프드내추럴 (중배전),아이템 요약 영역 입니다!!!,default_bean.png,0,F,상세 내용 입니다!!!,에티오피아,코케,G1,펄프드내추럴,8,8,7,4,5,5,중배전,"베리, 체리, 허니"
3,4,NaN,NaN,케냐 키암부 AA 워시드 (중배전),케냐 키암부 AA 워시드 (중배전),아이템 요약 영역 입니다!!!,default_bean.png,0,F,상세 내용 입니다!!!,케냐,키암부,AA,워시드,6,6,7,6,7,5,중배전,"자몽, 당밀, 카라멜"
4,5,NaN,NaN,콜롬비아 콜롬비아 SUPREMO 워시드 (중배전),콜롬비아 콜롬비아 SUPREMO 워시드 (중배전),아이템 요약 영역 입니다!!!,default_bean.png,0,F,상세 내용 입니다!!!,콜롬비아,콜롬비아,SUPREMO,워시드,7,5,6,6,6,5,중배전,"마일드, 적포도, 메이플시럽, 다크초코"


In [48]:
# TF-IDF 벡터화
tfidf_vector = TfidfVectorizer()
tfidf_matrix = tfidf_vector.fit_transform(bean_data['coffeeing_note']).toarray()
tfidf_matrix_feature = tfidf_vector.get_feature_names_out()

In [49]:
print(tfidf_matrix_feature)
print(tfidf_matrix)

['가벼운' '갈색설탕' '감귤' '감초' '강한' '개성전인' '건포도' '견과류' '고소한' '곡물' '과일' '과테말라'
 '균형있는' '균형잡힌' '깊은' '깔끔한' '다크초코' '달고나' '달콤판' '달콤한' '당밀' '딸기' '땅콩' '라임'
 '레드와인' '레몬' '마일드' '마일드한' '마카다미아' '맑은' '메이플시럽' '묵직한' '밀크초코' '바닐라' '밝은'
 '베리' '보리' '복숭아' '복잡한' '부드러운' '브라질' '블랙커런트' '블루마운틴' '사과' '사탕수수' '산뜻한'
 '산토스' '살구' '세이보리' '수프리모' '스모크' '스카치' '스카치캔디' '시트러스' '쌉쌀한' '아몬드' '아침'
 '안티구아' '에티오피아' '여운있는' '열대과일' '예가체프' '오렌지' '오크나무' '오트밀' '옥수수' '우아한' '자두'
 '자메이카' '자몽' '자연스러운' '적포도' '조청' '중후한' '진한' '청량한' '청포도' '체리' '초코' '카라멜'
 '카카오' '캔디' '코코넛밀크' '코코아' '콜롬비아' '쿠키' '크랜베리' '클래식' '토스트' '포도' '푸른사과' '풍부한'
 '플로럴' '피넛' '허니' '헤이즐넛' '현미' '호두파이' '호박고구마']
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.36150141 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         

In [50]:
%%time
# tfidf_matrix 기반 유사도 측정
grade_cosine_sim = cosine_similarity(tfidf_matrix)

CPU times: total: 15.6 ms
Wall time: 9.56 ms


In [52]:
%%time
# aroma ~ bady 까지의 스테이터스에 따른 유사도 특정
grade_cosine_sim = cosine_similarity(bean_data[['flavor', 'acidity', 'sweetness', 'bitterness', 'body']])

CPU times: total: 31.2 ms
Wall time: 2.26 ms


In [53]:
print(grade_cosine_sim.shape)
print(grade_cosine_sim.dtype)

grade_cosine_sim = grade_cosine_sim.astype(np.float16)
gc.collect()
print(grade_cosine_sim.dtype)

grade_cosine_sim

(47, 47)
float64
float16


array([[1.    , 0.902 , 0.999 , ..., 0.9355, 0.9214, 0.9214],
       [0.902 , 1.    , 0.8994, ..., 0.9785, 0.983 , 0.983 ],
       [0.999 , 0.8994, 1.    , ..., 0.9404, 0.9272, 0.9272],
       ...,
       [0.9355, 0.9785, 0.9404, ..., 1.    , 0.9927, 0.9927],
       [0.9214, 0.983 , 0.9272, ..., 0.9927, 1.    , 1.    ],
       [0.9214, 0.983 , 0.9272, ..., 0.9927, 1.    , 1.    ]],
      dtype=float16)

In [54]:
# 이름 뿐만 아니라 id로도 검색할 수 있도록 행과 열 중 하나를 title, 다른 하나를 id로 지정

df_grade_cosine_sim = pd.DataFrame(grade_cosine_sim, index = bean_data['idx'], columns = bean_data['name_ko'], dtype=np.float16)
print(df_grade_cosine_sim.shape)
df_grade_cosine_sim.head()

(47, 47)


name_ko,에티오피아 예가체프 G2 워시드 (중배전),과테말라 안티구아 SHB 워시드 (강배전),에티오피아 코케 G1 펄프드내추럴 (중배전),케냐 키암부 AA 워시드 (중배전),콜롬비아 콜롬비아 SUPREMO 워시드 (중배전),페루 피우라 MICRO LOT 워시드 (중배전),브라질 세하도 NY2 FC 내추럴 (중배전),케냐 키암부 FAQ 워시드 (중배전),파나마 보케테 SPECIALTY 워시드 (중배전),브라질 세하도 NY2 FC 펄프드내추럴 (중배전),브라질 세하도 SPECIALTY 펄프드내추럴 (중배전),코스타리카 나란조 MICRO LOT 워시드 (중배전),온두라스 엘 파라이소 SPECIALTY 워시드 (중배전),에티오피아 시다모 SPECIALTY 워시드 (중배전),과테말라 우에우에테낭고 SHB 워시드 디카페인 (강배전),콜롬비아 후일라 EXCELOS 워시드 디카페인 (강배전),인도 말라바르 AA 워시드 (중배전),코스타리카 따라주 SHB 워시드 (중배전),르완다 부산제 SPECIALTY 워시드 (중배전),브라질 카르모 데 미나스 - 워시드 디카페인 (중배전),엘살바도르 산타아 MICRO LOT 워시드 (중배전),인도네시아 수마트라 G1 웻헐드 (중배전),베트남 베트남 G1 워시드 (중배전),에티오피아 시다모 G2 워시드 (중배전),인도 치크마갈루르 AA 워시드 (중배전),에티오피아 시다모 G2 워시드 디카페인 (중배전),콜롬비아 콜롬비아 SUPREMO 워시드 디카페인 (강배전),에티오피아 예가체프 G2 워시드 (약배전),브라질 세하도 NY2 FC 네추럴 (중배전),과테말라 안티구아 SHB 워시드 (중배전),콜롬비아 후일라 SUPREMO 워시드 (중배전),베트남 다낭 G1 폴리싱 (강배전),브라질 산토스 NY2 FC 내추럴 (중배전),에티오피아 시다모 G4 내추럴 (중배전),에티오피아 예가체프 G4 내추럴 (중배전),탄자니아 음빙가 AA 워시드 (강배전),콜롬비아 메데인 SUPREMO 워시드 (강배전),코스타리카 따라주 SHB 워시드 (강배전),파푸아뉴기니 와기벨리 AA 워시드 (강배전),인도네시아 만델링 G1 워시드 (강배전),온두라스 산티아고 푸링글라 SHB 워시드 (강배전),엘 살바도르 엘 살바도르 SHG 워시드 (강배전),케냐 키암부 AA 워시드 (강배전),콜롬비아 메데인 SUPREMO 워시드 디카페인 (강배전),과테말라 안티구아 SHB 워시드 디카페인 (강배전),에티오피아 예가체프 G2 워시드 디카페인 (강배전),브라질 산토스 NY2 FC 워시드 디카페인 (강배전)
idx,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.901855,0.999023,0.962402,0.966309,0.963867,0.960449,0.971680,0.950195,0.967285,0.982422,0.993652,0.985352,0.996094,0.941895,0.927246,0.919434,0.979004,0.989746,0.969238,0.982910,0.907715,0.893066,0.983887,0.932129,0.964844,0.959961,0.985352,0.893555,0.918945,0.959961,0.973145,0.929199,0.984375,0.984375,0.935547,0.921387,0.953613,0.945312,0.921387,0.929199,0.953613,0.921387,0.921387,0.935547,0.921387,0.921387
2,0.901855,1.000000,0.899414,0.975586,0.979004,0.951660,0.976074,0.968262,0.990234,0.981934,0.942871,0.914551,0.948730,0.926270,0.990723,0.969238,0.984375,0.960449,0.937988,0.976074,0.962891,0.992188,0.996582,0.957520,0.993164,0.977051,0.967773,0.832520,0.983398,0.991699,0.967773,0.961426,0.984863,0.951172,0.951172,0.978516,0.982910,0.951660,0.981934,0.982910,0.984863,0.987305,0.982910,0.982910,0.978516,0.982910,0.982910
3,0.999023,0.899414,1.000000,0.962891,0.963867,0.971191,0.960938,0.969238,0.947754,0.965332,0.986328,0.997070,0.987793,0.996582,0.939453,0.930664,0.918945,0.974121,0.992676,0.969727,0.983398,0.904297,0.890625,0.983887,0.932129,0.967285,0.960938,0.989746,0.896484,0.921875,0.960938,0.974609,0.930176,0.984375,0.984375,0.940430,0.927246,0.954590,0.946777,0.927246,0.930176,0.954590,0.927246,0.927246,0.940430,0.927246,0.927246
4,0.962402,0.975586,0.962891,1.000000,0.991699,0.981445,1.000000,0.997070,0.993652,0.992676,0.976562,0.972168,0.988770,0.979980,0.995117,0.990234,0.965332,0.991699,0.986816,0.997559,0.994141,0.976074,0.961914,0.991699,0.990723,0.994629,0.988770,0.921875,0.955078,0.985840,0.988770,0.992676,0.993652,0.982910,0.982910,0.976562,0.982910,0.993164,0.984375,0.982910,0.993652,0.993164,0.982910,0.982910,0.976562,0.982910,0.982910
5,0.966309,0.979004,0.963867,0.991699,1.000000,0.977539,0.990234,0.994629,0.997070,0.998047,0.978516,0.968262,0.989258,0.979980,0.992188,0.966797,0.984375,0.992188,0.981934,0.996582,0.993652,0.984375,0.975098,0.994629,0.991699,0.994629,0.994141,0.917969,0.969727,0.978516,0.994141,0.982422,0.979004,0.992676,0.992676,0.980957,0.973145,0.971680,0.980469,0.973145,0.979004,0.996094,0.973145,0.973145,0.980957,0.973145,0.973145


In [55]:
df_grade_cosine_sim.loc[1].sort_values(ascending=False)[1:11].index

Index(['에티오피아 코케 G1 펄프드내추럴 (중배전)', '에티오피아 시다모 SPECIALTY 워시드 (중배전)',
       '코스타리카 나란조 MICRO LOT 워시드 (중배전)', '르완다 부산제 SPECIALTY 워시드 (중배전)',
       '에티오피아 예가체프 G2 워시드 (약배전)', '온두라스 엘 파라이소 SPECIALTY 워시드 (중배전)',
       '에티오피아 예가체프 G4 내추럴 (중배전)', '에티오피아 시다모 G4 내추럴 (중배전)',
       '에티오피아 시다모 G2 워시드 (중배전)', '엘살바도르 산타아 MICRO LOT 워시드 (중배전)'],
      dtype='object', name='name_ko')

In [56]:
df_grade_cosine_sim.index.get_indexer([1])

array([0], dtype=int64)

In [57]:
df_grade_cosine_sim.iloc[:, df_grade_cosine_sim.index.get_indexer([1])].columns[0]

'에티오피아 예가체프 G2 워시드 (중배전)'

In [58]:
df_grade_cosine_sim.iloc[:, df_grade_cosine_sim.index.get_indexer([1])].sort_values(by=df_grade_cosine_sim.iloc[:, df_grade_cosine_sim.index.get_indexer([1])].columns[0], ascending=False)[1:11].index

Int64Index([3, 14, 12, 19, 28, 13, 35, 34, 24, 21], dtype='int64', name='idx')

In [62]:
# id 기반 추천 알고리즘
def recommendations_by_id(target_id, matrix, items, k=10):
    try:
        target_idx = matrix.index.get_indexer([target_id])
        recom_idx = matrix.iloc[:, target_idx].sort_values(by= matrix.iloc[:, target_idx].columns[0], ascending=False)[1:11].index

        # 반환한 인덱스 값은 1부터 시작하나, 실제 iloc로 접근하는 인덱스 값은 0부터 시작하므로 이를 보정해야함
        recom_idx = recom_idx-1
        recom_id = items.iloc[recom_idx, :].idx.values
        recom_title = items.iloc[recom_idx, :].name_ko.values

        target_id_list = np.full(len(range(k)), target_id)
        target_title_list = np.full(len(range(k)), items[items.idx == target_id].name_ko.values)
        
    except:
        print(recom_idx)
        print(recom_id, recom_title)
        print(target_id_list, target_title_list)
    
    d = {
        'target_id': target_id_list,
        'target_title': target_title_list,
        'recom_id'    : recom_id,
        'recom_title' : recom_title,
    }
    
    return pd.DataFrame(d)

In [64]:
recommendations_by_id(3, df_grade_cosine_sim, bean_data)

,target_id,target_title,recom_id,recom_title
0,3,에티오피아 코케 G1 펄프드내추럴 (중배전),1,에티오피아 예가체프 G2 워시드 (중배전)
1,3,에티오피아 코케 G1 펄프드내추럴 (중배전),12,코스타리카 나란조 MICRO LOT 워시드 (중배전)
2,3,에티오피아 코케 G1 펄프드내추럴 (중배전),14,에티오피아 시다모 SPECIALTY 워시드 (중배전)
3,3,에티오피아 코케 G1 펄프드내추럴 (중배전),19,르완다 부산제 SPECIALTY 워시드 (중배전)
4,3,에티오피아 코케 G1 펄프드내추럴 (중배전),28,에티오피아 예가체프 G2 워시드 (약배전)
5,3,에티오피아 코케 G1 펄프드내추럴 (중배전),13,온두라스 엘 파라이소 SPECIALTY 워시드 (중배전)
6,3,에티오피아 코케 G1 펄프드내추럴 (중배전),11,브라질 세하도 SPECIALTY 펄프드내추럴 (중배전)
7,3,에티오피아 코케 G1 펄프드내추럴 (중배전),35,에티오피아 예가체프 G4 내추럴 (중배전)
8,3,에티오피아 코케 G1 펄프드내추럴 (중배전),34,에티오피아 시다모 G4 내추럴 (중배전)
9,3,에티오피아 코케 G1 펄프드내추럴 (중배전),24,에티오피아 시다모 G2 워시드 (중배전)


In [79]:
# id 기반 추천 알고리즘
def recommendation_list_by_id(target_id, matrix, items, k=10):
    try:
        target_idx =  matrix.index.get_indexer([target_id])
        recom_idx = matrix.iloc[:, target_idx].sort_values(by= matrix.iloc[:, target_idx].columns[0], ascending=False)[1:k+1].index
        
        # 반환한 인덱스 값은 1부터 시작하나, 실제 iloc로 접근하는 인덱스 값은 0부터 시작하므로 이를 보정해야함
        recom_idx = recom_idx-1
        recom_id = items.iloc[recom_idx, :].idx.values
        recom_title = items.iloc[recom_idx, :].name_ko.values
        
    except:
        print(recom_idx)
        print(recom_id, recom_title)
    
    recom_list = [dict(id = id, title = title) for id, title in zip(recom_id, recom_title)]
    
    return recom_list

In [81]:
recommendation_list_by_id(3, df_grade_cosine_sim, bean_data, k=5)

[{'id': 1, 'title': '에티오피아 예가체프 G2 워시드 (중배전)'},
 {'id': 12, 'title': '코스타리카 나란조 MICRO LOT 워시드 (중배전)'},
 {'id': 14, 'title': '에티오피아 시다모 SPECIALTY 워시드 (중배전)'},
 {'id': 19, 'title': '르완다 부산제 SPECIALTY 워시드 (중배전)'},
 {'id': 28, 'title': '에티오피아 예가체프 G2 워시드 (약배전)'}]

In [82]:
# 유사도 기준으로 추천 원두의 상위 5개를 출력
bean_recom = bean_data.copy()[['idx', 'name_ko']]
bean_recom['recommendation'] = bean_recom.apply(lambda x: recommendation_list_by_id(x.idx, df_grade_cosine_sim, bean_data, k=5), axis=1)
print(bean_recom.shape)
bean_recom.head()

(47, 3)


,idx,name_ko,recommendation
0,1,에티오피아 예가체프 G2 워시드 (중배전),"[{'id': 3, 'title': '에티오피아 코케 G1 펄프드내추럴 (중배전)'..."
1,2,과테말라 안티구아 SHB 워시드 (강배전),"[{'id': 23, 'title': '베트남 베트남 G1 워시드 (중배전)'}, ..."
2,3,에티오피아 코케 G1 펄프드내추럴 (중배전),"[{'id': 1, 'title': '에티오피아 예가체프 G2 워시드 (중배전)'}..."
3,4,케냐 키암부 AA 워시드 (중배전),"[{'id': 7, 'title': '브라질 세하도 NY2 FC 내추럴 (중배전)'..."
4,5,콜롬비아 콜롬비아 SUPREMO 워시드 (중배전),"[{'id': 10, 'title': '브라질 세하도 NY2 FC 펄프드내추럴 (중..."


In [83]:
# 파일 저장
os.makedirs(DIR_SAVE_PATH, exist_ok=True)
# bean_recom.to_csv(path.join(DIR_SAVE_PATH, 'bean_cbf_recom.csv'), sep=',')
bean_recom.to_csv(path.join(DIR_SAVE_PATH, 'bean_cbf_recom.csv'), sep=',', index=False)

In [84]:
# bean_cbf_recom.csv
recom_read = pd.read_csv(path.join(DIR_SAVE_PATH, 'bean_cbf_recom.csv'), low_memory=False)

print(recom_read.shape)
recom_read.head()

(47, 3)


,idx,name_ko,recommendation
0,1,에티오피아 예가체프 G2 워시드 (중배전),"[{'id': 3, 'title': '에티오피아 코케 G1 펄프드내추럴 (중배전)'..."
1,2,과테말라 안티구아 SHB 워시드 (강배전),"[{'id': 23, 'title': '베트남 베트남 G1 워시드 (중배전)'}, ..."
2,3,에티오피아 코케 G1 펄프드내추럴 (중배전),"[{'id': 1, 'title': '에티오피아 예가체프 G2 워시드 (중배전)'}..."
3,4,케냐 키암부 AA 워시드 (중배전),"[{'id': 7, 'title': '브라질 세하도 NY2 FC 내추럴 (중배전)'..."
4,5,콜롬비아 콜롬비아 SUPREMO 워시드 (중배전),"[{'id': 10, 'title': '브라질 세하도 NY2 FC 펄프드내추럴 (중..."


In [85]:
print(bean_read.shape)
bean_read.head()

(47, 8)


,idx,created_date,updated_date,name_ko,name_en,summary,thumbnail,user_grade
0,1,NaN,NaN,에티오피아 예가체프 G2 워시드 (중배전),에티오피아 예가체프 G2 워시드 (중배전),아이템 요약 영역 입니다!!!,default_bean.png,0
1,2,NaN,NaN,과테말라 안티구아 SHB 워시드 (강배전),과테말라 안티구아 SHB 워시드 (강배전),아이템 요약 영역 입니다!!!,default_bean.png,0
2,3,NaN,NaN,에티오피아 코케 G1 펄프드내추럴 (중배전),에티오피아 코케 G1 펄프드내추럴 (중배전),아이템 요약 영역 입니다!!!,default_bean.png,0
3,4,NaN,NaN,케냐 키암부 AA 워시드 (중배전),케냐 키암부 AA 워시드 (중배전),아이템 요약 영역 입니다!!!,default_bean.png,0
4,5,NaN,NaN,콜롬비아 콜롬비아 SUPREMO 워시드 (중배전),콜롬비아 콜롬비아 SUPREMO 워시드 (중배전),아이템 요약 영역 입니다!!!,default_bean.png,0


In [88]:
def get_recom_by_bean(itemIdx, matrix, k=5):
    try:
        recom_list = matrix.set_index('idx').loc[itemIdx]['recommendation']
        recom_list = json.loads(recom_list.replace('\'', '\"'))
        recom_list = [dict(t) for t in {tuple(d.items()) for d in recom_list}]
        
    except:
        print(itemIdx)
        print(recom_list)
        
    return recom_list[:k]

In [92]:
recom_list = get_recom_by_bean(1, recom_read)
recom_list

[{'id': 19, 'title': '르완다 부산제 SPECIALTY 워시드 (중배전)'},
 {'id': 28, 'title': '에티오피아 예가체프 G2 워시드 (약배전)'},
 {'id': 3, 'title': '에티오피아 코케 G1 펄프드내추럴 (중배전)'},
 {'id': 14, 'title': '에티오피아 시다모 SPECIALTY 워시드 (중배전)'},
 {'id': 12, 'title': '코스타리카 나란조 MICRO LOT 워시드 (중배전)'}]